# 01 - Data Cleaning: Job Datasets

This notebook loads and cleans all raw job datasets for the ArahBelajarAI skill demand forecasting pipeline.

**Datasets:**
1. `future_jobs_dataset.csv` (10,000 rows) - Emerging tech job postings
2. `jobs.csv` (779 rows) - India-focused job postings
3. `job_market.csv` (250 rows) - US/International tech jobs
4. `jobstreet_all_job_dataset.csv` (2M+ rows) - Malaysia/Singapore jobs
5. LinkedIn archive data - skill taxonomy & job-skill mappings

**Output:** Cleaned parquet files in `ml/data/process/`

## 1.1 Setup & Configuration

In [19]:
!pip install pandas numpy pyarrow fastparquet tqdm


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

RAW_DIR = Path('../data/raw')
JOB_DIR = RAW_DIR / 'job'
LINKEDIN_DIR = RAW_DIR / 'linkedin_archive'
PROCESS_DIR = Path('../data/process')
PROCESS_DIR.mkdir(parents=True, exist_ok=True)

print('Directories configured:')
print(f'  Raw data: {RAW_DIR.resolve()}')
print(f'  Job data: {JOB_DIR.resolve()}')
print(f'  LinkedIn: {LINKEDIN_DIR.resolve()}')
print(f'  Output:   {PROCESS_DIR.resolve()}')

Directories configured:
  Raw data: /Users/kharisma.wardhana/Documents/project/python/DB10-Capstone-ArahBelajarAI/ml/data/raw
  Job data: /Users/kharisma.wardhana/Documents/project/python/DB10-Capstone-ArahBelajarAI/ml/data/raw/job
  LinkedIn: /Users/kharisma.wardhana/Documents/project/python/DB10-Capstone-ArahBelajarAI/ml/data/raw/linkedin_archive
  Output:   /Users/kharisma.wardhana/Documents/project/python/DB10-Capstone-ArahBelajarAI/ml/data/process


In [3]:
# Helper function for data quality reporting
def data_quality_report(df, name):
    """Print a concise data quality summary for a DataFrame."""
    print(f'\n{"=" * 60}')
    print(f'Dataset: {name}')
    print(f'{"=" * 60}')
    print(f'Shape: {df.shape[0]:,} rows x {df.shape[1]} columns')
    print(f'\nColumn types:')
    for col in df.columns:
        null_count = df[col].isnull().sum()
        null_pct = null_count / len(df) * 100
        print(f'  {col:30s} {str(df[col].dtype):15s} nulls: {null_count:6d} ({null_pct:.1f}%)')
    
    dupes = df.duplicated().sum()
    print(f'\nExact duplicates: {dupes}')
    print(f'Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB')
    return {'name': name, 'rows': len(df), 'cols': len(df.columns), 'duplicates': dupes}

## 1.2 Clean `future_jobs_dataset.csv` (10K rows)

In [4]:
df_future = pd.read_csv(JOB_DIR / 'future_jobs_dataset.csv')
data_quality_report(df_future, 'future_jobs (raw)')


Dataset: future_jobs (raw)
Shape: 10,000 rows x 9 columns

Column types:
  job_id                         int64           nulls:      0 (0.0%)
  job_title                      object          nulls:      0 (0.0%)
  industry                       object          nulls:      0 (0.0%)
  location                       object          nulls:      0 (0.0%)
  salary_usd                     int64           nulls:      0 (0.0%)
  skills_required                object          nulls:      0 (0.0%)
  remote_option                  object          nulls:      0 (0.0%)
  company_size                   object          nulls:      0 (0.0%)
  posting_date                   object          nulls:      0 (0.0%)

Exact duplicates: 0
Memory usage: 4.7 MB


{'name': 'future_jobs (raw)',
 'rows': 10000,
 'cols': 9,
 'duplicates': np.int64(0)}

In [5]:
# Inspect value distributions
print('Industries:', df_future['industry'].value_counts().to_dict())
print('\nLocations:', df_future['location'].value_counts().to_dict())
print('\nRemote options:', df_future['remote_option'].value_counts().to_dict())
print('\nCompany sizes:', df_future['company_size'].value_counts().to_dict())
print(f'\nSalary range: ${df_future["salary_usd"].min():,.0f} - ${df_future["salary_usd"].max():,.0f}')
print(f'Date range: {df_future["posting_date"].min()} to {df_future["posting_date"].max()}')

Industries: {'Quantum Computing': 2519, 'Blockchain': 2499, 'AI': 2492, 'Green Tech': 2490}

Locations: {'New York': 1689, 'Singapore': 1682, 'Tokyo': 1673, 'Dubai': 1660, 'London': 1656, 'Berlin': 1640}

Remote options: {'Yes': 5089, 'No': 4911}

Company sizes: {'Large': 3385, 'Medium': 3328, 'Small': 3287}

Salary range: $50,013 - $249,990
Date range: 2025-01-01 to 2025-12-31


In [6]:
# Clean future_jobs
df_future_clean = df_future.copy()

# Drop duplicates on job_id
n_before = len(df_future_clean)
df_future_clean = df_future_clean.drop_duplicates(subset=['job_id'])
print(f'Duplicates removed: {n_before - len(df_future_clean)}')

# Parse posting_date to datetime
df_future_clean['posting_date'] = pd.to_datetime(df_future_clean['posting_date'])

# Convert remote_option to boolean
df_future_clean['remote_option'] = df_future_clean['remote_option'].map({'Yes': True, 'No': False})

# Convert company_size to ordered categorical
df_future_clean['company_size'] = pd.Categorical(
    df_future_clean['company_size'],
    categories=['Small', 'Medium', 'Large'],
    ordered=True
)

# Validate salary_usd
invalid_salary = df_future_clean['salary_usd'].isna() | (df_future_clean['salary_usd'] <= 0)
print(f'Invalid salaries: {invalid_salary.sum()}')

# Add source column
df_future_clean['source'] = 'future_jobs'

data_quality_report(df_future_clean, 'future_jobs (cleaned)')
df_future_clean.head()

Duplicates removed: 0
Invalid salaries: 0

Dataset: future_jobs (cleaned)
Shape: 10,000 rows x 10 columns

Column types:
  job_id                         int64           nulls:      0 (0.0%)
  job_title                      object          nulls:      0 (0.0%)
  industry                       object          nulls:      0 (0.0%)
  location                       object          nulls:      0 (0.0%)
  salary_usd                     int64           nulls:      0 (0.0%)
  skills_required                object          nulls:      0 (0.0%)
  remote_option                  bool            nulls:      0 (0.0%)
  company_size                   category        nulls:      0 (0.0%)
  posting_date                   datetime64[ns]  nulls:      0 (0.0%)
  source                         object          nulls:      0 (0.0%)

Exact duplicates: 0
Memory usage: 3.7 MB


,job_id,job_title,industry,location,salary_usd,skills_required,remote_option,company_size,posting_date,source
0,1,Quantum Researcher,Quantum Computing,Singapore,175780,"Linear Algebra, Quantum Algorithms",False,Large,2025-07-22,future_jobs
1,2,Renewable Energy Engineer,Green Tech,Singapore,137481,"Climate Data Analysis, Energy Modeling",True,Large,2025-09-26,future_jobs
2,3,Quantum Researcher,Quantum Computing,Tokyo,182081,"Linear Algebra, Qiskit",False,Medium,2025-12-31,future_jobs
3,4,Sustainability Analyst,Green Tech,Singapore,113822,"Climate Data Analysis, Energy Modeling",False,Large,2025-05-29,future_jobs
4,5,Smart Contract Engineer,Blockchain,London,92575,"Rust, Solidity",True,Small,2025-03-30,future_jobs


In [7]:
# NOTE: This dataset appears SYNTHETIC
# Evidence: near-uniform distribution across industries (~2500 each),
# locations (~1660 each), and exactly 2 skills per row.
# Temporal analysis from this dataset should be interpreted with caution.
print('Skills per row:', df_future_clean['skills_required'].str.count(',').add(1).value_counts().to_dict())
print('\nSample skills:', df_future_clean['skills_required'].head(10).tolist())

Skills per row: {2: 10000}

Sample skills: ['Linear Algebra, Quantum Algorithms', 'Climate Data Analysis, Energy Modeling', 'Linear Algebra, Qiskit', 'Climate Data Analysis, Energy Modeling', 'Rust, Solidity', 'Solidity, Rust', 'Climate Data Analysis, Energy Modeling', 'Qiskit, Quantum Algorithms', 'Climate Data Analysis, Energy Modeling', 'Ethereum, Solidity']


In [8]:
df_future_clean.to_parquet(PROCESS_DIR / 'cleaned_future_jobs.parquet', index=False)
print(f'Saved: cleaned_future_jobs.parquet ({len(df_future_clean):,} rows)')

Saved: cleaned_future_jobs.parquet (10,000 rows)


## 1.3 Clean `jobs.csv` (India, 779 rows)

In [9]:
df_india = pd.read_csv(JOB_DIR / 'jobs.csv')
data_quality_report(df_india, 'india_jobs (raw)')


Dataset: india_jobs (raw)
Shape: 780 rows x 8 columns

Column types:
  title                          object          nulls:      0 (0.0%)
  company                        object          nulls:      0 (0.0%)
  ratings                        float64         nulls:     34 (4.4%)
  reviews                        object          nulls:     34 (4.4%)
  experience                     object          nulls:      0 (0.0%)
  city                           object          nulls:      0 (0.0%)
  job_description                object          nulls:      0 (0.0%)
  skills                         object          nulls:      0 (0.0%)

Exact duplicates: 748
Memory usage: 0.5 MB


{'name': 'india_jobs (raw)',
 'rows': 780,
 'cols': 8,
 'duplicates': np.int64(748)}

In [10]:
df_india.head(3)

,title,company,ratings,reviews,experience,city,job_description,skills
0,Associate Staff Engineer (Data Science),Nagarro,3.9,4711 Reviews,5-7 Yrs,"Hybrid - Bangalore Rural, India",Bachelor s or master s degree in computer scie...,"NLP, Machine Learning on Azure, Data science o..."
1,Associate Staff Engineer (Data Science),Nagarro,3.9,4711 Reviews,5-7 Yrs,"Hybrid - Bangalore Rural, India",Bachelor s or master s degree in computer scie...,"NLP, Machine Learning on Azure, Data science o..."
2,AI Engineer,Naukri,NaN,NaN,6-10 Yrs,"Mumbai, Pune",Preferred Qualifications (Good to Have). Exper...,"Generative Ai, Machine Learning, Python, Natur..."


In [11]:
df_india_clean = df_india.copy()

# 1. Drop exact duplicates
n_before = len(df_india_clean)
df_india_clean = df_india_clean.drop_duplicates()
print(f'Exact duplicates removed: {n_before - len(df_india_clean)}')

# 2. Clean ratings: replace 'NA' string with NaN, convert to float
df_india_clean['ratings'] = pd.to_numeric(df_india_clean['ratings'], errors='coerce')
print(f'Ratings - valid: {df_india_clean["ratings"].notna().sum()}, null: {df_india_clean["ratings"].isna().sum()}')

# 3. Clean reviews: strip 'Reviews' suffix, convert to numeric
df_india_clean['reviews'] = (
    df_india_clean['reviews']
    .astype(str)
    .str.replace(r'\s*Reviews?', '', regex=True)
    .str.strip()
)
df_india_clean['reviews'] = pd.to_numeric(df_india_clean['reviews'], errors='coerce')

# 4. Parse experience: extract min/max years from strings like "5-7 Yrs"
exp_pattern = r'(\d+)-(\d+)\s*Yrs?'
exp_extracted = df_india_clean['experience'].str.extract(exp_pattern)
df_india_clean['experience_min'] = pd.to_numeric(exp_extracted[0], errors='coerce')
df_india_clean['experience_max'] = pd.to_numeric(exp_extracted[1], errors='coerce')

# 5. Clean city: split work mode and city name
def parse_city(city_str):
    if not isinstance(city_str, str):
        return pd.Series({'work_mode': np.nan, 'city_clean': np.nan})
    if 'Hybrid - ' in city_str:
        return pd.Series({'work_mode': 'Hybrid', 'city_clean': city_str.replace('Hybrid - ', '')})
    elif 'Remote' in city_str:
        return pd.Series({'work_mode': 'Remote', 'city_clean': city_str.replace('Remote - ', '')})
    else:
        return pd.Series({'work_mode': 'Onsite', 'city_clean': city_str})

city_parsed = df_india_clean['city'].apply(parse_city)
df_india_clean['work_mode'] = city_parsed['work_mode']
df_india_clean['city_clean'] = city_parsed['city_clean']

# 6. No date column - add placeholder
df_india_clean['posting_date'] = pd.NaT

# 7. Add source
df_india_clean['source'] = 'india_jobs'

data_quality_report(df_india_clean, 'india_jobs (cleaned)')
print(f'\nWork modes: {df_india_clean["work_mode"].value_counts().to_dict()}')

Exact duplicates removed: 748
Ratings - valid: 31, null: 1

Dataset: india_jobs (cleaned)
Shape: 32 rows x 14 columns

Column types:
  title                          object          nulls:      0 (0.0%)
  company                        object          nulls:      0 (0.0%)
  ratings                        float64         nulls:      1 (3.1%)
  reviews                        float64         nulls:      1 (3.1%)
  experience                     object          nulls:      0 (0.0%)
  city                           object          nulls:      0 (0.0%)
  job_description                object          nulls:      0 (0.0%)
  skills                         object          nulls:      0 (0.0%)
  experience_min                 int64           nulls:      0 (0.0%)
  experience_max                 int64           nulls:      0 (0.0%)
  work_mode                      object          nulls:      0 (0.0%)
  city_clean                     object          nulls:      0 (0.0%)
  posting_date             

In [12]:
df_india_clean.to_parquet(PROCESS_DIR / 'cleaned_india_jobs.parquet', index=False)
print(f'Saved: cleaned_india_jobs.parquet ({len(df_india_clean):,} rows)')

Saved: cleaned_india_jobs.parquet (32 rows)


## 1.4 Clean `job_market.csv` (250 rows)

In [13]:
df_market = pd.read_csv(JOB_DIR / 'job_market.csv')
data_quality_report(df_market, 'job_market (raw)')


Dataset: job_market (raw)
Shape: 250 rows x 10 columns

Column types:
  job_title                      object          nulls:      0 (0.0%)
  company                        object          nulls:      0 (0.0%)
  location                       object          nulls:      0 (0.0%)
  job_type                       object          nulls:     29 (11.6%)
  category                       object          nulls:     20 (8.0%)
  salary_min                     int64           nulls:      0 (0.0%)
  salary_max                     int64           nulls:      0 (0.0%)
  experience_required            float64         nulls:     43 (17.2%)
  publication_date               object          nulls:      0 (0.0%)
  skills                         object          nulls:     50 (20.0%)

Exact duplicates: 0
Memory usage: 0.1 MB


{'name': 'job_market (raw)',
 'rows': 250,
 'cols': 10,
 'duplicates': np.int64(0)}

In [14]:
df_market.head(3)

,job_title,company,location,job_type,category,salary_min,salary_max,experience_required,publication_date,skills
0,Engineering Manager,DataInc,"San Francisco, CA",Remote,Technology,151082,291345,4.0,2025-11-27,"AWS, Agile, Machine Learning, Kubernetes, Mong..."
1,Engineering Manager,EnterpriseHub,"New York, NY",Remote,Technology,156891,280075,3.0,2025-11-27,"Java, Agile, Git, SQL, Ruby, Go"
2,Engineering Manager,StartupXYZ,"Seattle, WA",Part-time,Technology,152134,280310,4.0,2025-11-27,"AWS, Python, Kubernetes, Git"


In [16]:
df_market_clean = df_market.copy()

# 1. Drop duplicates
n_before = len(df_market_clean)
df_market_clean = df_market_clean.drop_duplicates()
print(f'Duplicates removed: {n_before - len(df_market_clean)}')

# 2. Parse publication_date to datetime
df_market_clean['publication_date'] = pd.to_datetime(df_market_clean['publication_date'], errors='coerce')
print(f'Date range: {df_market_clean["publication_date"].min()} to {df_market_clean["publication_date"].max()}')
# NOTE: All rows have the same date (2025-11-27) - single snapshot, no temporal analysis possible

# 3. Validate salaries
invalid_salary = df_market_clean['salary_min'] > df_market_clean['salary_max']
print(f'Rows where salary_min > salary_max: {invalid_salary.sum()}')
if invalid_salary.any():
    # Swap if reversed
    mask = invalid_salary
    df_market_clean.loc[mask, ['salary_min', 'salary_max']] = (
        df_market_clean.loc[mask, ['salary_max', 'salary_min']].values
    )

# 4. Parse experience_required as int
df_market_clean['experience_required'] = pd.to_numeric(
    df_market_clean['experience_required'], errors='coerce'
).astype('Int64')

# 5. Normalize job_type
print(f'Job types: {df_market_clean["job_type"].value_counts().to_dict()}')

# 6. Add source
df_market_clean['source'] = 'job_market'

data_quality_report(df_market_clean, 'job_market (cleaned)')

Duplicates removed: 0
Date range: 2025-11-27 00:00:00 to 2025-11-27 00:00:00
Rows where salary_min > salary_max: 0
Job types: {'Remote': 55, 'Full-time': 50, 'Contract': 50, 'Part-time': 49, 'berufserfahren': 6, 'Full time': 3, 'Working student': 3, 'professional / experienced': 2, 'manager': 1, 'Internship': 1, 'berufseinstieg': 1}

Dataset: job_market (cleaned)
Shape: 250 rows x 11 columns

Column types:
  job_title                      object          nulls:      0 (0.0%)
  company                        object          nulls:      0 (0.0%)
  location                       object          nulls:      0 (0.0%)
  job_type                       object          nulls:     29 (11.6%)
  category                       object          nulls:     20 (8.0%)
  salary_min                     int64           nulls:      0 (0.0%)
  salary_max                     int64           nulls:      0 (0.0%)
  experience_required            Int64           nulls:     43 (17.2%)
  publication_date          

{'name': 'job_market (cleaned)',
 'rows': 250,
 'cols': 11,
 'duplicates': np.int64(3)}

In [17]:
df_market_clean.to_parquet(PROCESS_DIR / 'cleaned_job_market.parquet', index=False)
print(f'Saved: cleaned_job_market.parquet ({len(df_market_clean):,} rows)')

Saved: cleaned_job_market.parquet (250 rows)


## 1.5 Clean `jobstreet_all_job_dataset.csv` (2M+ rows, Memory-Critical)

**Strategy:** Chunked loading with `engine='python'` to handle multiline descriptions, then filter to tech-related categories only.

In [20]:
from tqdm.notebook import tqdm

# Tech-related categories to keep
TECH_CATEGORIES = {
    'Information & Communication Technology',
    'Engineering',
    'Science & Technology',
}

dtype_spec = {
    'job_id': 'float64',
    'job_title': 'str',
    'company': 'str',
    'descriptions': 'str',
    'location': 'str',
    'category': 'str',
    'subcategory': 'str',
    'role': 'str',
    'type': 'str',
    'salary': 'str',
    'listingDate': 'str',
}

print('Loading jobstreet dataset in chunks...')
print(f'Filtering to categories: {TECH_CATEGORIES}')

cleaned_chunks = []
total_raw = 0
total_kept = 0

try:
    chunks = pd.read_csv(
        JOB_DIR / 'jobstreet_all_job_dataset.csv',
        dtype=dtype_spec,
        chunksize=50_000,
        on_bad_lines='skip',
        engine='python',
    )
    
    for i, chunk in enumerate(chunks):
        total_raw += len(chunk)
        
        # Filter to tech categories
        chunk = chunk[chunk['category'].isin(TECH_CATEGORIES)]
        
        if len(chunk) == 0:
            continue
        
        # Drop rows with NaN job_id or empty job_title
        chunk = chunk.dropna(subset=['job_id', 'job_title'])
        
        # Convert job_id to int
        chunk['job_id'] = chunk['job_id'].astype(int)
        
        # Strip HTML tags from descriptions
        chunk['descriptions'] = chunk['descriptions'].fillna('').apply(
            lambda x: re.sub(r'<[^>]+>', ' ', str(x))
        )
        
        # Truncate very long descriptions
        chunk['descriptions'] = chunk['descriptions'].str[:5000]
        
        total_kept += len(chunk)
        cleaned_chunks.append(chunk)
        
        if (i + 1) % 5 == 0:
            print(f'  Chunk {i+1}: {total_raw:,} read, {total_kept:,} kept')

except Exception as e:
    print(f'Error during chunked read: {e}')

print(f'\nTotal rows read: {total_raw:,}')
print(f'Total tech rows kept: {total_kept:,}')

Loading jobstreet dataset in chunks...
Filtering to categories: {'Information & Communication Technology', 'Science & Technology', 'Engineering'}

Total rows read: 69,024
Total tech rows kept: 18,204


In [21]:
# Concatenate all chunks
df_jobstreet = pd.concat(cleaned_chunks, ignore_index=True)
del cleaned_chunks

# Drop exact duplicates on job_id
n_before = len(df_jobstreet)
df_jobstreet = df_jobstreet.drop_duplicates(subset=['job_id'])
print(f'Duplicates removed: {n_before - len(df_jobstreet)}')

# Parse listingDate to datetime
df_jobstreet['listingDate'] = pd.to_datetime(df_jobstreet['listingDate'], errors='coerce')
print(f'Date range: {df_jobstreet["listingDate"].min()} to {df_jobstreet["listingDate"].max()}')
print(f'Null dates: {df_jobstreet["listingDate"].isna().sum()}')

# Convert category to categorical for memory efficiency
for col in ['category', 'subcategory', 'role', 'type']:
    df_jobstreet[col] = df_jobstreet[col].astype('category')

# Add source
df_jobstreet['source'] = 'jobstreet'

data_quality_report(df_jobstreet, 'jobstreet (cleaned, tech only)')

Duplicates removed: 0
Date range: 2023-03-24 07:22:00+00:00 to 2025-05-08 10:16:46+00:00
Null dates: 0

Dataset: jobstreet (cleaned, tech only)
Shape: 18,204 rows x 12 columns

Column types:
  job_id                         int64           nulls:      0 (0.0%)
  job_title                      object          nulls:      0 (0.0%)
  company                        object          nulls:      0 (0.0%)
  descriptions                   object          nulls:      0 (0.0%)
  location                       object          nulls:      0 (0.0%)
  category                       category        nulls:      0 (0.0%)
  subcategory                    category        nulls:      0 (0.0%)
  role                           category        nulls:    675 (3.7%)
  type                           category        nulls:      0 (0.0%)
  salary                         object          nulls:  12507 (68.7%)
  listingDate                    datetime64[ns, UTC] nulls:      0 (0.0%)
  source                         o

{'name': 'jobstreet (cleaned, tech only)',
 'rows': 18204,
 'cols': 12,
 'duplicates': np.int64(0)}

In [22]:
# Inspect subcategories and roles within tech
print('Subcategories (top 20):')
print(df_jobstreet['subcategory'].value_counts().head(20))
print(f'\nRoles (top 15):')
print(df_jobstreet['role'].value_counts().head(15))

Subcategories (top 20):
subcategory
Electrical/Electronic Engineering    2348
Developers/Programmers               1843
Mechanical Engineering               1411
Management                           1344
Civil/Structural Engineering          969
Networks & Systems Administration     936
Help Desk & IT Support                934
Engineering - Software                837
Business/Systems Analysts             747
Programme & Project Management        572
Process Engineering                   486
Environmental Engineering             423
Supervisors                           411
Security                              355
Project Management                    327
Engineering Drafting                  326
Other                                 301
Testing & Quality Assurance           301
Automotive Engineering                292
Maintenance                           284
Name: count, dtype: int64

Roles (top 15):
role
engineer                            526
project-engineer                    

In [23]:
df_jobstreet.to_parquet(PROCESS_DIR / 'cleaned_jobstreet.parquet', index=False)
print(f'Saved: cleaned_jobstreet.parquet ({len(df_jobstreet):,} rows)')

Saved: cleaned_jobstreet.parquet (18,204 rows)


## 1.6 Clean LinkedIn Data (from archive.zip)

In [24]:
# LinkedIn Skills Taxonomy (35 categories)
df_linkedin_skills = pd.read_csv(LINKEDIN_DIR / 'mappings' / 'skills.csv')
print('LinkedIn Skill Categories:')
print(df_linkedin_skills)
print(f'\nTotal categories: {len(df_linkedin_skills)}')

LinkedIn Skill Categories:
   skill_abr              skill_name
0        ART            Art/Creative
1       DSGN                  Design
2       ADVR             Advertising
3       PRDM      Product Management
4       DIST            Distribution
5        EDU               Education
6       TRNG                Training
7       PRJM      Project Management
8       CNSL              Consulting
9       PRCH              Purchasing
10      SUPL            Supply Chain
11      ANLS                 Analyst
12      HCPR    Health Care Provider
13      RSCH                Research
14       SCI                 Science
15      GENB        General Business
16      CUST        Customer Service
17      STRA       Strategy/Planning
18       FIN                 Finance
19      OTHR                   Other
20       LGL                   Legal
21       ENG             Engineering
22        QA       Quality Assurance
23        BD    Business Development
24        IT  Information Technology
25       AD

In [25]:
# LinkedIn Job-Skills Mapping (213K rows)
df_linkedin_job_skills = pd.read_csv(LINKEDIN_DIR / 'jobs' / 'job_skills.csv')
data_quality_report(df_linkedin_job_skills, 'linkedin_job_skills (raw)')

# Drop duplicates
n_before = len(df_linkedin_job_skills)
df_linkedin_job_skills = df_linkedin_job_skills.drop_duplicates()
print(f'\nDuplicates removed: {n_before - len(df_linkedin_job_skills)}')

# Merge skill names
df_linkedin_job_skills = df_linkedin_job_skills.merge(
    df_linkedin_skills, on='skill_abr', how='left'
)

print(f'\nSkill distribution in LinkedIn jobs:')
print(df_linkedin_job_skills['skill_name'].value_counts().head(15))


Dataset: linkedin_job_skills (raw)
Shape: 213,768 rows x 2 columns

Column types:
  job_id                         int64           nulls:      0 (0.0%)
  skill_abr                      object          nulls:      0 (0.0%)

Exact duplicates: 0
Memory usage: 13.9 MB

Duplicates removed: 0

Skill distribution in LinkedIn jobs:
skill_name
Information Technology    26137
Sales                     22475
Management                20861
Manufacturing             18185
Health Care Provider      17369
Business Development      14290
Engineering               13009
Other                     12608
Finance                    8540
Marketing                  5525
Accounting/Auditing        5461
Administrative             4860
Customer Service           4292
Project Management         3997
Analyst                    3858
Name: count, dtype: int64


In [26]:
# Save LinkedIn data
df_linkedin_skills.to_parquet(PROCESS_DIR / 'linkedin_skills_taxonomy.parquet', index=False)
df_linkedin_job_skills.to_parquet(PROCESS_DIR / 'linkedin_job_skills.parquet', index=False)
print(f'Saved: linkedin_skills_taxonomy.parquet ({len(df_linkedin_skills)} rows)')
print(f'Saved: linkedin_job_skills.parquet ({len(df_linkedin_job_skills):,} rows)')

Saved: linkedin_skills_taxonomy.parquet (35 rows)
Saved: linkedin_job_skills.parquet (213,768 rows)


## 1.7 Data Quality Summary

In [27]:
summary = pd.DataFrame([
    {
        'Dataset': 'future_jobs',
        'Rows (raw)': 10_000,
        'Rows (clean)': len(df_future_clean),
        'Columns': len(df_future_clean.columns),
        'Has Skills': True,
        'Has Dates': True,
        'Date Range': f"{df_future_clean['posting_date'].min().strftime('%Y-%m')} to {df_future_clean['posting_date'].max().strftime('%Y-%m')}",
        'Has Salary': True,
    },
    {
        'Dataset': 'india_jobs',
        'Rows (raw)': 779,
        'Rows (clean)': len(df_india_clean),
        'Columns': len(df_india_clean.columns),
        'Has Skills': True,
        'Has Dates': False,
        'Date Range': 'N/A',
        'Has Salary': False,
    },
    {
        'Dataset': 'job_market',
        'Rows (raw)': 250,
        'Rows (clean)': len(df_market_clean),
        'Columns': len(df_market_clean.columns),
        'Has Skills': True,
        'Has Dates': True,
        'Date Range': '2025-11 (single date)',
        'Has Salary': True,
    },
    {
        'Dataset': 'jobstreet',
        'Rows (raw)': 2_057_213,
        'Rows (clean)': len(df_jobstreet),
        'Columns': len(df_jobstreet.columns),
        'Has Skills': False,
        'Has Dates': True,
        'Date Range': f"{df_jobstreet['listingDate'].min()} to {df_jobstreet['listingDate'].max()}",
        'Has Salary': True,
    },
    {
        'Dataset': 'linkedin_job_skills',
        'Rows (raw)': 213_000,
        'Rows (clean)': len(df_linkedin_job_skills),
        'Columns': len(df_linkedin_job_skills.columns),
        'Has Skills': True,
        'Has Dates': False,
        'Date Range': 'N/A',
        'Has Salary': False,
    },
])

print('\n' + '=' * 80)
print('DATA QUALITY SUMMARY - ALL DATASETS')
print('=' * 80)
display(summary)

# List output files
print('\nOutput files in ml/data/process/:')
for f in sorted(PROCESS_DIR.glob('*.parquet')):
    size_mb = f.stat().st_size / 1024**2
    print(f'  {f.name:45s} {size_mb:.1f} MB')


DATA QUALITY SUMMARY - ALL DATASETS


,Dataset,Rows (raw),Rows (clean),Columns,Has Skills,Has Dates,Date Range,Has Salary
0,future_jobs,10000,10000,10,True,True,2025-01 to 2025-12,True
1,india_jobs,779,32,14,True,False,N/A,False
2,job_market,250,250,11,True,True,2025-11 (single date),True
3,jobstreet,2057213,18204,12,False,True,2023-03-24 07:22:00+00:00 to 2025-05-08 10:16:...,True
4,linkedin_job_skills,213000,213768,3,True,False,N/A,False



Output files in ml/data/process/:
  cleaned_future_jobs.parquet                   0.2 MB
  cleaned_india_jobs.parquet                    0.0 MB
  cleaned_job_market.parquet                    0.0 MB
  cleaned_jobstreet.parquet                     20.9 MB
  linkedin_job_skills.parquet                   1.2 MB
  linkedin_skills_taxonomy.parquet              0.0 MB
